In [3]:
import torch
from torch import nn
net=nn.Sequential(nn.Linear(4,8),nn.ReLU(),nn.Linear(8,1))
X=torch.rand(size=(2,4))
net(X)


tensor([[0.2722],
        [0.3379]], grad_fn=<AddmmBackward0>)

In [8]:
print(net[0].state_dict()['weight'])
print(net[2].state_dict())#net[2]是第三层，因为net[0]是第一层，net[1]是第二层,state_dict()是一个字典，包含了权重和偏差


tensor([[ 0.4618,  0.3711, -0.4284,  0.4671],
        [-0.3664,  0.3480, -0.4116, -0.3064],
        [ 0.4131, -0.3684,  0.3903, -0.4995],
        [-0.3987,  0.1118,  0.2031, -0.4070],
        [-0.1891, -0.1764,  0.0973,  0.3953],
        [ 0.1984,  0.1443,  0.0735, -0.3891],
        [ 0.0285,  0.3913, -0.2087,  0.1239],
        [-0.2119,  0.2970, -0.4895,  0.0037]])
OrderedDict([('weight', tensor([[ 0.2158, -0.2549,  0.1865,  0.2554, -0.2058,  0.1254,  0.0439, -0.0224]])), ('bias', tensor([0.1986]))])


In [10]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)
print(net[2].weight)
print(net[2].weight.data)
print(net[2].weight.grad)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([0.1986], requires_grad=True)
tensor([0.1986])
Parameter containing:
tensor([[ 0.2158, -0.2549,  0.1865,  0.2554, -0.2058,  0.1254,  0.0439, -0.0224]],
       requires_grad=True)
tensor([[ 0.2158, -0.2549,  0.1865,  0.2554, -0.2058,  0.1254,  0.0439, -0.0224]])
None


In [11]:
#一次性访问所有参数
print(*[(name,param.shape) for name,param in net[0].named_parameters()])#named_parameters()返回一个字典，包含了权重和偏差,这里的name是字典的键，param是字典的值,这里的*是解包
print(*[(name,param.shape) for name,param in net.named_parameters()])


('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [13]:
net.state_dict()['2.bias'].data#这里的bias是net[2]的偏差，weight是net[2]的权重

tensor([[ 0.2158, -0.2549,  0.1865,  0.2554, -0.2058,  0.1254,  0.0439, -0.0224]])

In [ ]:
net.state_dict()['2.weight'].data

In [14]:
#从嵌套收集参数
def block1():
    return nn.Sequential(nn.Linear(4,8),nn.ReLU(),nn.Linear(8,4),nn.ReLU())

def block2():
    net=nn.Sequential()
    for i in range(4):
        net.add_module(f'block{i}',block1())
    return net
rgnet=nn.Sequential(block2(),nn.Linear(4,1))

rgnet(X)

tensor([[-0.4493],
        [-0.4493]], grad_fn=<AddmmBackward0>)

In [15]:
print(rgnet)


Sequential(
  (0): Sequential(
    (block0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [17]:
#内置初始化
def init_normal(m):
    if type(m)==nn.Linear:
        nn.init.normal_(m.weight,mean=0,std=0.01)
        nn.init.zeros_(m.bias)
net.apply(init_normal)
print(net[0].weight.data)
print(net[0].weight.data[0])#
# net[0].weight.data[0],
# net[0].bias.data[0]


tensor([[ 0.0099, -0.0031, -0.0105,  0.0070],
        [-0.0027, -0.0046,  0.0002, -0.0088],
        [ 0.0089, -0.0212, -0.0057, -0.0210],
        [ 0.0057, -0.0181,  0.0040,  0.0044],
        [-0.0011, -0.0040,  0.0124, -0.0128],
        [-0.0042, -0.0093,  0.0143, -0.0137],
        [-0.0058,  0.0033,  0.0023,  0.0187],
        [-0.0139,  0.0143, -0.0042,  0.0189]])
tensor([ 0.0099, -0.0031, -0.0105,  0.0070])


In [20]:
def init_constant(m):
    if type(m)==nn.Linear:
        nn.init.constant_(m.weight,1)
        nn.init.zeros_(m.bias)

net.apply(init_constant)
print(net[0].weight)
print(net[0].weight.data[0])
net[0].bias.data[0]

Parameter containing:
tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]], requires_grad=True)
tensor([1., 1., 1., 1.])


tensor(0.)

In [21]:
def xavier(m):
    if type(m)==nn.Linear:
        nn.init.xavier_uniform_(m.weight)

def init_42(m):
    if type(m)==nn.Linear:
        nn.init.constant_(m.weight,42)

net[0].apply(xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([-0.1499, -0.3412,  0.6051,  0.4053])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


In [ ]:
#参数绑定
shared=nn.Linear(8,8)
net=nn.Sequential(nn.Linear(4,8),nn.ReLU(),shared,nn.ReLU(),shared,nn.ReLU(),nn.Linear(8,1))
net(X)

In [ ]:
print(net[2].weight.data[0]==net[4].weight.data[0])
net[2].weight.data[0,0]=100
print(net[2].weight.data[0]==net[4].weight.data[0])
